In [ ]:
import os
import json
import torch

from json import JSONDecodeError
import numpy as np
from tqdm import tqdm
from PIL import Image
from thumbnail_repr_stats import ImageLatentRepresentationModel, load_model, get_latent_vectors
from collections import defaultdict
from util.constants import Topic

In [ ]:
# Models

def model(vid):
    return

Get latents for all videos

In [ ]:
# Read data
videos = []
for cat in Topic._member_names_:
    with open(os.path.join("..", "data", "info_videos", F"videos-info_{cat}.json"), "r") as f:
        videos_info = json.load(f)
        videos.extend([vid for channel_vids in videos_info.values() for vid in channel_vids])

In [ ]:
RESULTS_DIR = os.path.join("..", "data", "thumbnail-latents")

video_results_dir = os.path.join(RESULTS_DIR, "videos")
channel_results_dir = os.path.join(RESULTS_DIR, "channels")
categories_results_dir = os.path.join(RESULTS_DIR, "categories")

if not os.path.exists(video_results_dir):
    os.makedirs(video_results_dir)

if not os.path.exists(categories_results_dir):
    os.makedirs(categories_results_dir)

def get_done_list(dir):
    return [nm.replace(".json",'') for nm in os.listdir(dir)]

In [ ]:
def generate_repr_stats(video_path):
    """
    Function to generate all thumbnail latent representation statistics.

    args:
        - video_path: path to the thumbnail that we want to load
    """

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}\n")

    model, feature_extractor = load_model(device, install=False)

    # Print model size
    mem_params = sum([param.nelement()*param.element_size() for param in model.parameters()])
    mem_bufs = sum([buf.nelement()*buf.element_size() for buf in model.buffers()])
    print(f"Memory used by model: {mem_params + mem_bufs} bytes")

    video_thumbnail = Image.open(video_path)
    latents = get_latent_vectors(model, feature_extractor, video_thumbnail, device)
    latents_save = latents.detach().cpu().numpy().tolist()

    return latents_save


In [ ]:
# Calculate video results
vid2result = {}
# done_list = get_done_list(video_results_dir)
for vid in tqdm(videos):
    id = vid["id"]
    # if id in done_list:
    #     continue
    img_path = "../data/thumbnails/"+id+"_high.jpg"
    if os.path.isfile(img_path):
        try:
            vid2result[id] = generate_repr_stats(img_path, id, video_results_dir)
        except:
            print("Video path not found")
            continue

In [ ]:
# Save video results
for vid_id,result in vid2result.items():
    path = os.path.join(video_results_dir, f"{vid_id}.json")
    with open(path, "w") as f:
        json.dump(result, f)

Channel stats

In [ ]:
# Read data
with open(os.path.join("..", "data", "vid2channel.json"), "r") as f:
    vid2channel = json.load(f)

In [ ]:
# Make list of results per video for each channel
channel_result_list = defaultdict(list)
for vid_id in tqdm(get_done_list(video_results_dir)):
    channel = vid2channel[vid_id]
    filepath = os.path.join(video_results_dir, f"{vid_id}.json")
    try:
        with open(filepath, "r") as f:
            result = json.load(f)
    except JSONDecodeError:
        print(f"couldn't open {vid_id}; deleting file")
        os.remove(filepath)
    channel_result_list[channel].append(result)

In [ ]:
# Calculate channel results
channel_results = {}
for channel,result_list in channel_result_list.items():
    result_list = np.array(result_list)
    channel_results[channel] = {
        "mean": result_list.mean(axis=0).tolist(),
        "std": result_list.std(),
        "len": len(result_list),
    }

In [ ]:
# Save channel stats
for channel,results in channel_results.items():
    filepath = os.path.join(channel_results_dir, f"{channel}.json")
    with open(filepath, "w") as f:
        json.dump(results, f)

Category stats

In [ ]:
# Read data
with open(os.path.join("..", "data", "channel2category.json"), "r") as f:
    channel2cat = json.load(f)

In [ ]:
# Make list of results per channel for each category
category_results_list = defaultdict(list)
for channel in tqdm(get_done_list(channel_results_dir)):
    cat = channel2cat[channel]
    filepath = os.path.join(channel_results_dir, f"{channel}.json")
    try:
        with open(filepath, "r") as f:
            results = json.load(f)
    except JSONDecodeError:
        print(f"couldn't open {channel}; deleting file")
        os.remove(filepath)
    category_results_list[cat].append(results)

In [ ]:
# Calculate category results
category_results = {}
for cat,stats_list in category_results_list.items():
    mean_list = np.array([channel_stats["mean"] for channel_stats in stats_list])
    std_list = np.array([channel_stats["std"] for channel_stats in stats_list])
    category_results[cat] = {
        "mean": mean_list.mean(axis=0).tolist(),
        "std": std_list.mean(),
        "len": len(mean_list),
    }

In [ ]:
# Save category results
for cat,stats in category_results.items():
    filepath = os.path.join(RESULTS_DIR, "categories", f"{cat}.json")
    with open(filepath, "w") as f:
        json.dump(stats, f)